### Load Temporally Streaming Tensor

In [270]:
import numpy as np
import tensorly as tl
from tensorly.decomposition import parafac

In [79]:
def construct_tensor(factors):
    weights = tl.ones(factors[0].shape[1])
    est_tensor = tl.kruskal_to_tensor((weights, factors))
    return est_tensor
    
def print_tensor(X):
    print(np.round(X, 1))
    
def compare_tensors(A, B):
    print('||A-B||:', tl.norm(A - B))

### CP-ALS in 3rd dimension

In [90]:
from tensorly.decomposition.candecomp_parafac import initialize_factors, unfolding_dot_khatri_rao, KruskalTensor

def cp_als_3rd(X, rank, n_iter=100, verbose=False):
        
    factors = initialize_factors(X, rank, init='svd', svd='numpy_svd',
                                 random_state=None,
                                 non_negative=False,
                                 normalize_factors=False)
    weights = tl.ones(rank)

    A = factors[0]
    B = factors[1]
    C = factors[2]
    for i in range(n_iter):
        V = tl.dot(tl.transpose(C), C)*tl.dot(tl.transpose(B), B)
        mttkrp = unfolding_dot_khatri_rao(X, (None, [A, B, C]), 0)
        A = tl.transpose(tl.solve(tl.transpose(V), tl.transpose(mttkrp)))
        
        V = tl.dot(tl.transpose(C), C)*tl.dot(tl.transpose(A), A)
        mttkrp = unfolding_dot_khatri_rao(X, (None, [A, B, C]), 1)
        B = tl.transpose(tl.solve(tl.transpose(V), tl.transpose(mttkrp)))
        
        V = tl.dot(tl.transpose(B), B)*tl.dot(tl.transpose(A), A)
        mttkrp = unfolding_dot_khatri_rao(X, (None, [A, B, C]), 2)
        C = tl.transpose(tl.solve(tl.transpose(V), tl.transpose(mttkrp)))
        
        if verbose:
            X_est = construct_tensor([A, B, C])
            compare_tensors(X, X_est)
    
    return KruskalTensor((weights, [A, B, C]))

def cp_als(X, rank, n_iter=100, verbose=False):
    factors = initialize_factors(X, rank, init='svd', svd='numpy_svd',
                                 random_state=None,
                                 non_negative=False,
                                 normalize_factors=False)
    weights = tl.ones(rank)
    n_dim = tl.ndim(X)
    
    for i in range(n_iter):
        for mode in range(n_dim):
            V = tl.tensor(np.ones((rank, rank)))
            for j, factor in enumerate(factors):
                if j != mode:
                    V = V * tl.dot(tl.transpose(factor), factor)
                    
            mttkrp = unfolding_dot_khatri_rao(X, (None, factors), mode)
            factors[mode] = tl.transpose(tl.solve(tl.transpose(V), tl.transpose(mttkrp)))
        
        if verbose:
            X_est = construct_tensor(factors)
            compare_tensors(X, X_est)
    
    return KruskalTensor((weights, factors))

### OnlineCP in 3rd dimension 
*transformed from original updating A0*

In [253]:
def online_cp_als_3rd(factors_old, X_old, X_new, rank, n_iter=100, verbose=False):
    weights = tl.ones(rank)
    A_old, B_old, C_old = factors_old
    X = np.concatenate((X_old, X_new))
    
    A0 = A_old
    B = B_old
    C = C_old
    
    for i in range(n_iter):
        V = tl.dot(tl.transpose(C), C)*tl.dot(tl.transpose(B), B)
        mttkrp = unfolding_dot_khatri_rao(X_new, (None, [A0, B, C]), 0)
        A1 = tl.transpose(tl.solve(tl.transpose(V), tl.transpose(mttkrp)))

        V = tl.dot(tl.transpose(C), C)*(tl.dot(tl.transpose(A0), A0) + tl.dot(tl.transpose(A1), A1))
        mttkrp0 = unfolding_dot_khatri_rao(X_old, (None, [A0, B, C]), 1)
        mttkrp1 = unfolding_dot_khatri_rao(X_new, (None, [A1, B, C]), 1)
        B = tl.transpose(tl.solve(tl.transpose(V), tl.transpose(mttkrp0 + mttkrp1)))

        V = tl.dot(tl.transpose(B), B)*(tl.dot(tl.transpose(A0), A0) + tl.dot(tl.transpose(A1), A1))
        mttkrp0 = unfolding_dot_khatri_rao(X_old, (None, [A0, B, C]), 2)
        mttkrp1 = unfolding_dot_khatri_rao(X_new, (None, [A1, B, C]), 2)
        C = tl.transpose(tl.solve(tl.transpose(V), tl.transpose(mttkrp0 + mttkrp1)))
        
        V = tl.dot(tl.transpose(C), C)*tl.dot(tl.transpose(B), B)
        mttkrp = unfolding_dot_khatri_rao(X_old, (None, [A0, B, C]), 0)
        A0 = tl.transpose(tl.solve(tl.transpose(V), tl.transpose(mttkrp)))
        
        A = np.concatenate((A0, A1))

        if verbose:
            X_est = construct_tensor([A, B, C])
            compare_tensors(X, X_est)

    return KruskalTensor((weights, [A, B, C]))

def online_cp_als(factors_old, X_old, X_new, rank, n_iter=100, verbose=False):
    weights = tl.ones(rank)
    X = tl.tensor(np.concatenate((X_old, X_new)))
    n_dim = tl.ndim(X)
    U = factors_old.copy()
    
    for i in range(n_iter):
        # temporal mode for A1
        V = tl.tensor(np.ones((rank, rank)))
        for j, factor in enumerate(U):
            if j != 0:
                V = V * tl.dot(tl.transpose(factor), factor)
        mttkrp = unfolding_dot_khatri_rao(X_new, (None, U), 0)
        A1 = tl.transpose(tl.solve(tl.transpose(V), tl.transpose(mttkrp)))

        # non-temporal mode
        for mode in range(1, n_dim):
            U1 = U.copy()
            U1[0] = A1
            V = tl.tensor(np.ones((rank, rank)))
            for j, factor in enumerate(U):
                if j != mode:
                    if j == 0:
                        V = V * (tl.dot(tl.transpose(factor), factor) + tl.dot(tl.transpose(A1), A1))
                    else:
                        V = V * tl.dot(tl.transpose(factor), factor)
            mttkrp0 = unfolding_dot_khatri_rao(X_old, (None, U), mode)
            mttkrp1 = unfolding_dot_khatri_rao(X_new, (None, U1), mode)
            U[mode] = tl.transpose(tl.solve(tl.transpose(V), tl.transpose(mttkrp0 + mttkrp1)))

            
        # temporal mode for A0
        V = tl.tensor(np.ones((rank, rank)))
        for j, factor in enumerate(U):
            if j != 0:
                V = V * tl.dot(tl.transpose(factor), factor)
        mttkrp = unfolding_dot_khatri_rao(X_old, (None, U), 0)
        U[0] = tl.transpose(tl.solve(tl.transpose(V), tl.transpose(mttkrp)))

        if verbose:
            U1 = U.copy()
            U1[0] = np.concatenate((U[0], A1))
            X_est = construct_tensor(U1)
            compare_tensors(X, X_est)

    U[0] = np.concatenate((U[0], A1))
    return KruskalTensor((weights, U))

### DTD in 3rd dimension

In [281]:
def dtd_als_3rd(factors_old, X_old, X_new, rank, mu=1, n_iter=100, verbose=False):
    weights = tl.ones(rank)
    A_old, B_old, C_old = factors_old
    X = np.concatenate((X_old, X_new))
        
    A0 = A_old
    B = B_old
    C = C_old
    
    for i in range(n_iter):
        V = tl.dot(tl.transpose(C), C)*tl.dot(tl.transpose(B), B)
        mttkrp = unfolding_dot_khatri_rao(X_new, (None, [A0, B, C]), 0)
        A1 = tl.transpose(tl.solve(tl.transpose(V), tl.transpose(mttkrp)))
        
        V = tl.dot(tl.transpose(C), C)*(mu*tl.dot(tl.transpose(A0), A0) + tl.dot(tl.transpose(A1), A1))
        mttkrp0 = mu*tl.dot(B_old, tl.dot(tl.transpose(C_old), C)*(tl.dot(tl.transpose(A_old), A0)))
        mttkrp1 = unfolding_dot_khatri_rao(X_new, (None, [A1, B, C]), 1)
        B = tl.transpose(tl.solve(tl.transpose(V), tl.transpose(mttkrp0 + mttkrp1)))

        V = tl.dot(tl.transpose(B), B)*(mu*tl.dot(tl.transpose(A0), A0) + tl.dot(tl.transpose(A1), A1))
        mttkrp0 = mu*tl.dot(C_old, tl.dot(tl.transpose(B_old), B)*(tl.dot(tl.transpose(A_old), A0)))
        mttkrp1 = unfolding_dot_khatri_rao(X_new, (None, [A1, B, C]), 2)
        C = tl.transpose(tl.solve(tl.transpose(V), tl.transpose(mttkrp0 + mttkrp1)))
        
        V = tl.dot(tl.transpose(C), C)*tl.dot(tl.transpose(B), B)
        mttkrp = tl.dot(A_old, tl.dot(tl.transpose(C_old), C)*tl.dot(tl.transpose(B_old), B))
        A0 = tl.transpose(tl.solve(tl.transpose(V), tl.transpose(mttkrp)))
        
        A = np.concatenate((A0, A1))

        if verbose:
            X_est = construct_tensor([A, B, C])
            compare_tensors(X, X_est)
    
    return KruskalTensor((weights, [A, B, C]))

def dtd_als(factors_old, X_old, X_new, rank, mu=1, n_iter=100, verbose=False):
    weights = tl.ones(rank)
    X = np.concatenate((X_old, X_new))
    n_dim = tl.ndim(X)
    U = factors_old.copy()
    
    for i in range(n_iter):
        # temporal mode for A1
        V = tl.tensor(np.ones((rank, rank)))
        for j, factor in enumerate(U):
            if j != 0:
                V = V * tl.dot(tl.transpose(factor), factor)
        mttkrp = unfolding_dot_khatri_rao(X_new, (None, U), 0)
        A1 = tl.transpose(tl.solve(tl.transpose(V), tl.transpose(mttkrp)))

        
        # non-temporal mode
        for mode in range(1, n_dim):
            U1 = U.copy()
            U1[0] = A1
            V = tl.tensor(np.ones((rank, rank)))
            W = tl.tensor(np.ones((rank, rank)))
            for j, factor in enumerate(U):
                factor_old = factors_old[j]
                if j != mode:
                    W = W * tl.dot(tl.transpose(factor_old), factor)
                    if j == 0:
                        V = V * (mu*tl.dot(tl.transpose(factor), factor) + tl.dot(tl.transpose(A1), A1))
                    else:
                        V = V * tl.dot(tl.transpose(factor), factor)
            mttkrp0 = mu * tl.dot(factors_old[mode], W)
            mttkrp1 = unfolding_dot_khatri_rao(X_new, (None, U1), mode)
            U[mode] = tl.transpose(tl.solve(tl.transpose(V), tl.transpose(mttkrp0 + mttkrp1)))

        # temporal mode for A0
        V = tl.tensor(np.ones((rank, rank)))
        W = tl.tensor(np.ones((rank, rank)))
        for j, factor in enumerate(U):
            factor_old = factors_old[j]
            if j != 0:
                V = V * tl.dot(tl.transpose(factor), factor)
                W = W * tl.dot(tl.transpose(factor_old), factor)
        mttkrp = tl.dot(factors_old[0], W)
        U[0] = tl.transpose(tl.solve(tl.transpose(V), tl.transpose(mttkrp)))

        if verbose:
            U1 = U.copy()
            U1[0] = np.concatenate((U[0], A1))
            X_est = construct_tensor(U1)
            compare_tensors(X, X_est)

    U[0] = np.concatenate((U[0], A1))
    return KruskalTensor((weights, U))

#### Testing for an example tensor

In [290]:
import time
tensor = tl.tensor(np.arange(40000000, dtype='d').reshape((50000, 40, 20)))
X_old = tensor[:30000,:,:]
X_new = tensor[30000:,:,:]
rank = 2

# cp-als: 0.400312
X = np.concatenate((X_old, X_new))
start = time.time()
print('\n >> cp_als')
(weights, factors) = parafac(X, rank, verbose=False)
print('exec time:', time.time()-start)
X_est = construct_tensor(factors)
compare_tensors(X, X_est)
# print_tensor(X_est[:, 0, 0])

# online_cp: 0.731866
start = time.time()
print('\n >> online_cp')
(weights, factors) = parafac(X_old, rank)
init_time = time.time()
print('init time:', init_time-start)
(weights, factors) = online_cp_als(factors, X_old, X_new, rank, n_iter=10, verbose=True)
print('exec time:', time.time()-init_time)
X_est = construct_tensor(factors)
compare_tensors(X, X_est)
# print_tensor(X_est[:, 0, 0])

# dtd: 0.723(mu=1) 0.722692(mu=0.8)
start = time.time()
print('\n >> dtd')
(weights, factors) = parafac(X_old, rank)
init_time = time.time()
print('init time:', init_time-start)
(weights, factors) = dtd_als(factors, X_old, X_new, rank, mu=1, n_iter=10, verbose=True)
print('exec time:', time.time()-init_time)
X_est = construct_tensor(factors)
compare_tensors(X, X_est)
# print_tensor(X_est[:, 0, 0])



 >> cp_als
exec time: 2.7576189041137695
||A-B||: 729786.9681705366

 >> online_cp
init time: 1.7549586296081543
||A-B||: 98285.7268219792
||A-B||: 18063.395715669412
||A-B||: 18063.242338002132
||A-B||: 18063.08458961561
||A-B||: 18062.926695303744
||A-B||: 18062.768655093485
||A-B||: 18062.61046912696
||A-B||: 18062.452136876866
||A-B||: 18062.293658378472
||A-B||: 18062.13503336239
exec time: 18.222131967544556
||A-B||: 18062.13503336239

 >> dtd
init time: 1.838104009628296
||A-B||: 567154.9806718915
||A-B||: 565177.5619175966
||A-B||: 565177.5449783339
||A-B||: 565177.5280396888
||A-B||: 565177.5110863777
||A-B||: 565177.4941182547
||A-B||: 565177.4771353502
||A-B||: 565177.4601377293
||A-B||: 565177.4431253802
||A-B||: 565177.4260983495
exec time: 12.444217920303345
||A-B||: 565177.4260983495
